<a href="https://colab.research.google.com/github/minsungkim1017/pyconsulting/blob/main/file_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**작업개요 (엑셀 시트 합치기)**

1.   여러 개의 엑셀 파일에서 특정 시트를 검색하여 내용을 병합함
2.   각 시트의 열 이름 및 구조가 상이한 경우에도 가능함

---

초기 파일경로는 개인 구글드라이브 내 "data_합치기" 폴더로 설정되어 있음
폴더 내 xlsx 이외의 파일이 포함되어도 작동하지만, 권장하지 않음

결과물은 data가 저장된 폴더 내 "merge.xlsx"로 저장됨

전체 폴더 내 엑셀 파일 수 = A + B + C

1.   전체 엑셀파일 중에서 특정 시트명을 포함하지 않은 경우(A)

2.   특정 시트명을 포함하였으나, 구조적인 문제 등 로 병합에 실패한 경우 (B)

3.   자료병합에 성공한 경우 (C)



파일병합이 실패하는 원인: 구조적인 문제, 해당 시트가 공백인 경우


---


주의사항:
1. 엑셀파일에 수식 형태로 남아 있는 경우 계산된 수식 값이 아니라 셀 주소형식으로 가져옴
2. 시트 내 별도의 집계 열(소계, 합계) 또는 메모가 있는 경우도 전체를 가져옴
3. merge 파일에서 별도의 처리 작업을 해야할 수 있음 

last update: 22년 2월 28일 (MS KIM)

In [1]:
#필요한 라이브러리 로딩하기

import os
import openpyxl
import pandas as pd

In [2]:
# 작업파일이 있는 폴더에 접근하기

os.chdir("/content/drive/MyDrive/data_합치기")
file_list = os.listdir()
num_file = len(file_list)
print(str(num_file) +  "개 파일이 존재합니다.")

162개 파일이 존재합니다.


In [3]:
# 확장자가 엑셀 파일인 것들만 골라서 리스트 만들기

xlsx_name = []
non_xlsx_name = []
for file in file_list:
  if file.endswith(".xlsx"):
    xlsx_name.append(str(file))

  else:
    non_xlsx_name.append(str(file))

print(str(len(xlsx_name)) + "개 엑셀 파일이 발견되었습니다.")

df = pd.DataFrame({'file':xlsx_name}).reset_index()
df['index'] = df['index'].astype('str')
df.head()

162개 엑셀 파일이 발견되었습니다.


,index,file
0,0,200427 20년04월 물류수수료_영림산업.xlsx
1,1,200502 20년04월 물류수수료_가구만드는남ᄌ...
2,2,200502 20년04월 물류수수료_누리다(누리고그ᄅ...
3,3,200502 20년04월 물류수수료_노송퍼니처.xlsx
4,4,200502 20년04월 물류수수료_꾸밈(애니모리)....


In [4]:
#특정 키워드/시트명이 존재하는 파일을 찾아서 리스트로 만들기

keyword = input("시트 검색 키워드를 입력하세요.")

file_idx = []
true_list = []

for i in range(0,len(df)):
    wb = openpyxl.load_workbook(str(df.iloc[i,1]))
    for sht in wb.sheetnames:
        if keyword in sht:
            file_idx.append(str(i))
            true_list.append(sht)
        

df1 = pd.DataFrame({"file_index":file_idx,
                   "sheet_name": true_list})

print()
print(keyword + " 검색결과" + "총" + str(len(df1)) + "개 파일이 발견되었습니다.")

시트 검색 키워드를 입력하세요.시공비

시공비 검색결과총157개 파일이 발견되었습니다.


In [5]:
#해당 시트명에 첫 줄이 열 이름일 경우를 가정함

df2 = pd.merge(df1, df, left_on = "file_index", right_on = "index")
df2 = df2.drop(['index'], axis = 1)

print("작업용 DF 생성완료")
df2.head()

result = pd.DataFrame()
error_list = []

for i in range(0, len(df2)):
  try:
    wb = openpyxl.load_workbook(df2.iloc[i,2])
    sht = wb[df2.iloc[i,1]]
    data = sht.values
    col = next(data)[0:]
    sht_df = pd.DataFrame(data, columns=col)
    sht_df['file'] = str(df2.iloc[i,2])
    result = pd.concat([result, sht_df], axis = 0)
    if i % 10 == 0:
      print(str(i) + "번째까지 성공")


  except:
    error_list.append(str(df2.iloc[i,2]))
    print(str(i) + "번째 실패")

result.to_excel("merge.xlsx")
print("merge.xlsx 저장")


작업용 DF 생성완료
0번째까지 성공


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})


10번째까지 성공
12번째 실패
20번째까지 성공
30번째까지 성공
40번째까지 성공
50번째 실패
60번째까지 성공
70번째까지 성공
78번째 실패
80번째까지 성공
87번째 실패
89번째 실패
90번째까지 성공
100번째까지 성공
110번째까지 성공
120번째까지 성공
124번째 실패
130번째까지 성공
140번째까지 성공
142번째 실패
150번째 실패
merge.xlsx 저장


In [6]:
#에러 리스트 반환

error_list

['200502 20년04월 물류수수료_드림메이드_창고비일할.xlsx',
 '200502 20년04월 물류수수료_브라운즈.xlsx',
 '200502 20년04월 물류수수료_오럭스_창고비일할.xlsx',
 '200502 20년04월 물류수수료_제이케이디파트너스.xlsx',
 '200502 20년04월 물류수수료_인퍼니.xlsx',
 '200502 20년04월 물류수수료_에스엠엘디자인(SML) v3.xlsx',
 '200502 20년04월 물류수수료_하늘디자인 v4.xlsx',
 '200502 20년04월 물류수수료_넵스홈.xlsx']

In [7]:
#해당 키워드를 포함하지 않아서 제외된 목록

include_list = pd.merge(df, df1, left_on = "index", right_on= "file_index")
include_idx = list(include_list.index)

df_omit = df.copy()
df_omit.drop(include_idx, inplace=True)
df_omit
# df_omit.to_excel("작업제외항목.xlsx")
# print("작업제외항목.xlsx를 생성했습니다.")

,index,file
157,157,200502 20년04월 물류수수료_아파트맨터리.xlsx
158,158,200502 20년04월 물류수수료_페로나.xlsx
159,159,200427 20년04월 물류수수료_에넥스.xlsx
160,160,200502 20년04월 물류수수료_레스티.xlsx
161,161,200502 20년04월 물류수수료_온라인투어v3(ᄋ...
